In [4]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# IMPORT YOUR CRUD MODULE
from CRUD_Python_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "admin"

# Connect to database via CRUD Module
db = AnimalShelter(username, password)

# Load entire dataset
df = pd.DataFrame.from_records(db.read({}))
df.drop(columns=['_id'], inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Logo
image_filename = 'Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([

    html.Center(html.Img(
        src='data:image/png;base64,{}'.format(encoded_image.decode()),
        style={'width': '200px'}
    )),

    html.Center(html.B(html.H1('ZDavis583190957'))),
    html.Hr(),

    # FILTERING OPTIONS
    html.Div([
        html.Label("Select Rescue Type:"),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'All Animals', 'value': 'all'},
                {'label': 'Water Rescue', 'value': 'water'},
                {'label': 'Mountain / Wilderness Rescue', 'value': 'mountain'},
                {'label': 'Disaster / Individual Tracking', 'value': 'disaster'}
            ],
            value='all',
            inline=True
        )
    ]),

    html.Hr(),

    # DATA TABLE
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        page_action="native",
        page_current=0,
        page_size=10,
        row_selectable="single",
        selected_rows=[],
        style_cell={'textAlign': 'left', 'padding': '5px'},
        style_header={'backgroundColor': 'lightgrey', 'fontWeight': 'bold'}
    ),

    html.Br(),
    html.Hr(),

    # CHART + MAP SIDE BY SIDE
    html.Div(className='row',
             style={'display': 'flex'},
             children=[
                html.Div(id='graph-id', className='col s12 m6'),
                html.Div(id='map-id', className='col s12 m6')
             ]
    )
])


#############################################
# Interaction Between Components / Controller
#############################################


# FILTERING LOGIC
@app.callback(Output('datatable-id','data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):

    if filter_type == "water":
        query = {
            "animal_type": "Dog",
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
            "breed": {"$in": [
                "Labrador Retriever Mix",
                "Chesapeake Bay Retriever",
                "Newfoundland"
            ]}
        }

    elif filter_type == "mountain":
        query = {
            "animal_type": "Dog",
            "sex_upon_outcome": "Intact Male",
            "breed": {"$in": [
                "German Shepherd",
                "Alaskan Malamute",
                "Old English Sheepdog",
                "Siberian Husky",
                "Rottweiler"
            ]}
        }

    elif filter_type == "disaster":
        query = {
            "animal_type": "Dog",
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300},
            "breed": {"$in": [
                "Doberman Pinscher",
                "German Shepherd",
                "Golden Retriever",
                "Bloodhound",
                "Rottweiler"
            ]}
        }

    else:
        query = {}

    newdf = pd.DataFrame(list(db.read(query)))
    if '_id' in newdf.columns:
        newdf.drop(columns=['_id'], inplace=True)

    return newdf.to_dict('records')



# BREED PIE CHART
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")]
)
def update_graphs(viewData):

    if viewData is None or len(viewData) == 0:
        return [html.H3("No data available")]

    dff = pd.DataFrame.from_dict(viewData)

    return [
        dcc.Graph(
            figure=px.pie(
                data_frame=dff,
                names='breed',
                title='Breed Distribution'
            )
        )
    ]


# MAP
@app.callback(
    Output('map-id', "children"),
    [
        Input('datatable-id', "derived_virtual_data"),
        Input('datatable-id', "derived_virtual_selected_rows")
    ]
)
def update_map(viewData, index):

    if viewData is None or index is None or len(index) == 0:
        return [html.H3("Select an animal from the table")]

    dff = pd.DataFrame.from_dict(viewData)

    row = index[0]
    lat = dff.iloc[row]['location_lat']
    lon = dff.iloc[row]['location_long']
    breed = dff.iloc[row]['breed']
    name = dff.iloc[row]['name']

    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[lat, lon], zoom=12, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(breed),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(name)
                ])
            ])
        ])
    ]


# STYLE SELECTED COLUMN
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# Run app
app.run_server()


Dash app running on https://domainvalery-rachelgood-3000.codio.io/proxy/8050/
